# Flow Detection

This notebook is used to design and play with the flow detection algorithm.

In [27]:
# Global imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys
sys.path.append('../swiss_flows')

## Data Cleaning

Clean the data: 

In [2]:
from clean_tweets import clean_tweets

# clean_tweets('../data/clean_tweets')

Import the clean tweets: 

In [3]:
tweets = pd.read_csv('../data/clean_tweets.csv', parse_dates=[2])
tweets.head()

,id,userId,createdAt,placeLongitude,placeLatitude,userLocation
0,776522983837954049,735449229028675584,2016-09-15 20:48:01,8.96044,46.0027,Earleen.
1,776523000636203010,2741685639,2016-09-15 20:48:05,8.22414,46.8131,Suisse
2,776523045200691200,435239151,2016-09-15 20:48:15,5.94082,47.2010,Fontain
3,776523058404290560,503244217,2016-09-15 20:48:18,6.16552,45.8011,Shargeyah
4,776523058504925185,452805259,2016-09-15 20:48:18,6.14319,46.2048,İstanbul/Burgazada


## Grouping by user

In order to detect flows, we need to analyse the differents locations of people. This requires to analyse tweets by user.

In [4]:
# Group by user
grouped = tweets.groupby('userId')

nb_user = len(tweets['userId'].value_counts())
print('Number of different users : {}.'.format(nb_user))

Number of different users : 2763.


How many tweets by user do we get ?

*Note: The following cells have been commented because they cause the evaluation of the notebook (Run All) to be slow.*

In [5]:
'''
df = grouped.agg('count')['id'].reset_index().rename(columns={'id': 'tweets'}, index=str)
df.head()
'''

"\ndf = grouped.agg('count')['id'].reset_index().rename(columns={'id': 'tweets'}, index=str)\ndf.head()\n"

In [6]:
'''
df['tweets'].value_counts().plot(kind='bar')
plt.title('Distribution of tweets per user')
plt.xlabel('Number of tweets')
plt.ylabel('Number of user')
plt.show()
'''

"\ndf['tweets'].value_counts().plot(kind='bar')\nplt.title('Distribution of tweets per user')\nplt.xlabel('Number of tweets')\nplt.ylabel('Number of user')\nplt.show()\n"

Well, it seems most of the users tweeted only once. This isn't very good for our ultimate goal. But we have to keep in mind that the actual dataset contains much more tweets, and so much more user that tweet more than once.

## Filter users

Users who only post a unique tweet provide no insight in the flows we wish to detect. Their corresponding tweets should be removed.

How many users do we initially have?

In [7]:
len(grouped)

2763

Let's take a look at the data, grouped by user.

In [8]:
# grouped is a DataFrameGroupBy object, it cannot be displayed like a dataframe
"""for name, group in grouped:
    print(name)
    print(group)
    print('--------------------------------------------------------------------')
"""

"for name, group in grouped:\n    print(name)\n    print(group)\n    print('--------------------------------------------------------------------')\n"

- We filter out the users who only have 1 one tweet. 
- The object we currently have is a DataFrameGroupBy object, which is unusual and hard to manipulate. Hence, we convert it to a dictionary.
- Since the keys of this dictionary correspond to the userId, we no longer need the ```userId``` column.

In [9]:
# Transform the DataFrameGroupBy object into a dictionary
user_tweets = {}
for name, group in grouped:
    
    # Filter out users with less than 1 tweet
    if(group.shape[0] > 1):
        
        # Remove the userId column since we use it as key
        user_tweets[name] = group.drop('userId', axis=1)
        
#user_tweets

## Flow detection

### Time interval condition

Tweets which are excessively spaced out in time are not viable. In other words, tweet emitted at time $t$ by a given user should be removed if this user hasn't emitted another tweet within the interval $[t - l/2, t + l/2]$, where $l$ is a set duration.

Therefore, for each user, we create all pairs of tweets which respect the above criterion, since they may represent a potential flow. We remove tweets which cannot pair with another tweet from the same user.

### Node condition

The pairs generated represent potential flows only if the tweets were emitted from different Nodes. Consequently, we remove pairs where:
- one or both tweet(s) do(es) not reside in a Node
- both tweets were emitted from the same Node

### Symmetrical flows
If flows are not directed : A <--> B is equivalent to B <--> A. We normalize undirected flows by lexicographical order of their nodes **in the constructor**.

Directed flows can't be normalized as they define a precise direction.

### Overlapping flows

Note that there is a problem here. Some users have an incredible number of pairs which potentially represent flows. However, a lot of those pairs represent the exact same flow. Indeed, certain users post several tweets at the same location. Therefore, a user may have several pairs representing the same flow, during the same period. We should only keep one of those pairs.

#### Example 

Consider this sequence of tweet locations : A -> A -> B with the 2 first tweets being posted in the same hour. If we naively consider all tweet pairs we will generate 2 undirected flows A -> B and A -> B, which actually represent the same logical tweet.

A solution could be the following : each time we generate a flow, we associate to it the time of the 2 tweets we used. If an identical flow (or symmetrical for undirected flows) overlaps, we should just keep one of those flows.

#### Problem

There is something we need to notice we trying to detect overlapping flows, consider the following time sequence: `A1 - B1 - A2 - B2`, with `Ax` and `Bx` representing the locations `A` and `B`.

Depending on the order we treat the tweet pairs, we will get different results: 
* `A1 - B1` then `B1 - A2`... finally `A1 - B2`: the 3 small intervals will be counted as **3 different moves** as they don't overlap, however the last one `A1-B2` won't be count since it overlaps with the others.
* `A1 - B2` first, then the others: the first big one will be counted as **1 move**, and the others will be ignored as they overlap with the first interval.
* We could probably find other results with other orders...

So we have an ordering problem here.

**Proposed solution**:

By looking at our example, it is clear that in our context, the true version we want to get is the first one, as moves with smaller intervals logically represent a flow and moves with bigger intervals represent a flow only if they don't overlap with smaller ones.

The idea is to simply treat small intervals first. Then when it comes to bigger ones, we will count them only if they don't overlap with small ones and get a deterministic result.

This is done by sorting the list of pairs for each user by interval length, and going through this list in order. 

### Weighting flows
Of course if two identical flows don't overlap, they shouldn't be considered are the exact same flow in time, so we should attribute a greater weight.

### Directed flows

The problem here is to try to identify if flows have directions, i.e. instead of saying that people are moving _between_ A and B, being able to say that people are moving _from_ A _to_ B.

#### Strategy

Consider the following sequence `A -> B -> A -> B`.

What should we conclude about the flows ? We identify two strategies: 

1. Someone is moving from A to B, and we should only count the flow `A -> B`.
2. Someone is moving from A to B and from B to A, and we should count both `A -> B` and `B -> A`.

**Strategy 1**:

This strategy seems to be the most logical one, since in this example we intuitively see that A is the starting point. But this is actually wrong, it turns out that even if we have a tweet from A first, it doesn't mean that the person is starting from A. Imagine we have one tweet from B right before : `B -> A -> B -> A -> B`, it completely changes the meaning we consider this strategy...

**Strategy 2**:

This is the naive strategy, i.e., basically counting the pairs of tweets and associating the earliest one to the starting point.

One can ask itself how does this fundamentally change from undirected flows, as we just decompose flows naively. Well, there are mutliple things to consider:

* If we count 2 flows `A -> B` and only 1 `B -> A`, it might be that some other flows, let's say `C -> D` comes with an weight of 2, and makes it more important than `B -> A`, which couldn't happen with undirected flows.
* We can also bring rafinements to the detection process, for example with the pattern above, if we have `B (11pm) -> A (8am)`, we can maybe deduce that B is actually not the starting point.

#### Implementation

The strategy which seems the most reasonnable is the second one, with the following choices: 

* The starting point is infered from the tweet that preceeds the other.
* A tweet preceeds another if it was posted logically before in time **and** it was posted earlier considering only the time of the day.

### Code

In [10]:
from node import Node

# Generate the nodes
nodes = Node.generate_nodes(n_swiss_nodes=10, n_foreign_nodes=1, pop_threshold=15000) 

../swiss_flows/node.py:262: UserWarning: File already exists, importing ...
  warnings.warn('File already exists, importing ...', UserWarning)


In [11]:
def by_interval_len(tweet_tuple):
    tmp1 = tweet_tuple[0]
    tmp2 = tweet_tuple[1]
    
    # Order the tweet by timestamp
    t1 = tmp1 if tmp1[1].to_pydatetime() < tmp2[1].to_pydatetime() else tmp2
    t2 = tmp2 if tmp1[1].to_pydatetime() < tmp2[1].to_pydatetime() else tmp1
    
    # Return the length of the interval
    return t2[1].to_pydatetime() - t1[1].to_pydatetime()

In [37]:
from flow import Flow
import itertools

# Duration in days
l = 2

# Detect directed or undirect flows
directed = True

user_flows = {}
for user_id, tweet_info in user_tweets.items():
    
    # Generate all possible pairs of tweet sorted by interval length
    pairs = sorted(list(itertools.combinations(tweet_info.values.tolist(), 2)), key=by_interval_len)

    # {f1 : {weight:1, intervals:[interval1, interval2...]}}
    flows = {}
    for id_pair in pairs:
        
        # [id, Timestamp, lon, lat, location]
        t1 = id_pair[0]
        t2 = id_pair[1]
                
        # Nodes corresponding to the tweets
        n1 = Node.locate_point((t1[3], t1[2]), nodes)
        n2 = Node.locate_point((t2[3], t2[2]), nodes)
                
        # Time interval condition
        time1 = t1[1].to_pydatetime()
        time2 = t2[1].to_pydatetime()
        ts1 = time1 if time1 < time2 else time2
        ts2 = time2 if time1 < time2 else time1
        tweet_interval = (ts1, ts2)
        time_cond = (ts2 - ts1).days <= l
        
        # Node conditions
        geo_cond = n1 and n2 and (n1 != n2)
                
        if time_cond and geo_cond:
            # Build the flow
            src = n1
            dst = n2
            
            if directed: 
                if time1 < time2 and time1.time() < time2.time():
                    src = n1
                    dst = n2
                elif time2 < time1 and time2.time() < time1.time():
                    src = n2
                    dst = n1
                else:
                    # Cannot conclude
                    continue
                    
            flow = Flow(src=src, dst=dst, directed=directed)
                        
            overlap = False
            if flow in flows:
                # Look for overlapping flows
                for interval in flows[flow]['intervals']:
                    if Flow.is_overlapping(tweet_interval, interval):
                        overlap = True
                        break
            
            else:
                # Add the initial values if it's a new flow
                flows[flow] = {'weight': 1, 'intervals': [], 'start':ts1, 'end':ts2}
            
            # If no overlap, then it's not the exact same flow
            if not overlap:
                # Update start date
                flows[flow]['start'] = min(ts1, flows[flow]['start'])
                
                # Update end date
                flows[flow]['end'] = max(ts2, flows[flow]['end'])
                
                # Update weight
                flows[flow]['weight'] += 1
                
            # In any case, add the interval we just found for later use
            flows[flow]['intervals'].append(tweet_interval)
    
    # Save those flows
    if len(flows) > 0:
        user_flows[user_id] = flows

We can get rid of the interval lists and associate each flow with its weight:

In [38]:
print('{}% of the users define a flow.'.format(len(user_flows)*100/len(grouped)))

1.339124140427072% of the users define a flow.


This is pretty low... but we can expect to get something more reliable with the whole set of tweets. 

## Aggregating flows
We need to aggregate flows infered from different users in order to define the importance of each flow, and update the dates.

**NOTE**: we can integrate this step in the detection phase.

We tried to aggregate flows using `pandas` as it's probably more efficient. It turns out we need ordering amont flows which doesn't really make sense here. Given the amount of flows is limited, it's still reasonnable to aggregate greedily.

In [39]:
# Aggregate the flows
agg_flows = {}

for user, flows in user_flows.items():
    for flow, attr in flows.items():
        if flow not in agg_flows:
            agg_flows[flow] = {'weight': attr['weight'], 'start': attr['start'], 'end': attr['end']}
        else:
            agg_flows[flow]['weight'] += attr['weight']
            agg_flows[flow]['start'] = min(agg_flows[flow]['start'], attr['start'])
            agg_flows[flow]['end'] = min(agg_flows[flow]['end'], attr['end'])

In [40]:
# Update the weight attribute of each flow
final_flows = []
for flow, attr in agg_flows.items():
    flow.weight = attr['weight']
    flow.start_date = attr['start']
    flow.end_date = attr['end']
    final_flows.append(flow)
    
# Sort it by weight
final_flows.sort(key=lambda x: x.weight, reverse=True)

In [41]:
print('{} flows.'.format(len(final_flows)))

for flow in final_flows:
    print(flow)

57 flows.
[Flow] Bern --> Winterthur (weight: 36, start: 2016-09-16 02:05:04, end: 2016-09-16 03:45:05).
[Flow] Bern --> Geneve (weight: 11, start: 2016-09-16 02:05:04, end: 2016-09-16 11:07:16).
[Flow] Lausanne --> Geneve (weight: 10, start: 2016-09-16 00:25:46, end: 2016-09-16 08:52:09).
[Flow] Geneve --> Zurich (weight: 9, start: 2016-09-16 06:08:16, end: 2016-09-16 09:30:25).
[Flow] Bern --> Basel (weight: 8, start: 2016-09-16 02:05:04, end: 2016-09-16 04:45:14).
[Flow] Luzern --> Zurich (weight: 7, start: 2016-09-16 05:21:47, end: 2016-09-16 10:31:17).
[Flow] Zurich --> Luzern (weight: 7, start: 2016-09-16 05:50:59, end: 2016-09-16 07:55:03).
[Flow] Zurich --> Winterthur (weight: 7, start: 2016-09-16 08:13:25, end: 2016-09-16 13:39:51).
[Flow] Basel --> Zurich (weight: 6, start: 2016-09-16 04:45:14, end: 2016-09-16 12:35:53).
[Flow] Bern --> Zurich (weight: 6, start: 2016-09-16 02:05:04, end: 2016-09-16 13:36:30).
[Flow] Lausanne --> Bern (weight: 6, start: 2016-09-16 09:33:00, en